<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/01_EDA_%26_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ аномалий в начислениях за тепловую энергию


## 1. Анализ и подготовка данных



In [1]:
initiate = True
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

Mounted at /content/drive
/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import datetime

import numpy as np
import pandas as pd

from data.make_dataset import load_data
from data.preprocess import Preprocess

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'
file_names = os.listdir(folder_path)

file_names

['Апрель 2022 Уфа.xls',
 'Ноябрь 2022 Уфа.xlsx',
 'Июнь 2022 Уфа.xls',
 'Июнь 2023 Уфа.xlsx',
 'Сентябрь 2021 Уфа.xls',
 'Температуры, продолжительность ОП.xls',
 'Февраль 2022 Уфа.xls',
 'Январь 2023 Уфа.xlsx',
 'Январь 2022 Уфа.xls',
 'Май 2023 Уфа.xlsx',
 'Сентябрь 2022 Уфа.xlsx',
 'Июль 2022 Уфа.xlsx',
 'Тип строения, этажность, площадь, год постройки.xlsx',
 'Декабрь 2022 Уфа.xlsx',
 'Примеры аномалий.docx',
 'Ноябрь 2021 Уфа.xls',
 'Август 2021 Уфа.xls',
 'Март 2023 Уфа.xlsx',
 'Июль 2021 Уфа.xls',
 'Май 2022 Уфа.xls',
 'Декабрь 2021 Уфа.xls',
 'Октябрь 2021 Уфа.xls',
 'Октябрь 2022 Уфа.xlsx',
 'Март 2022 Уфа.xls',
 'Август 2022 Уфа.xlsx',
 'Февраль 2023 Уфа.xlsx',
 '.gitkeep',
 'Апрель 2023 Уфа.xlsx']

In [7]:
regenerate = True
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:11<00:00,  2.10it/s]


In [8]:
print(f"data.shape = {data.shape}\n")
data.sample(2)

data.shape = (75391, 9)



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
45523,4065,Уфа,200847,NaN,"г Уфа, ул. Цветочная, д.7 корп.3",Другое строение,2023-04-01,13.063,2023-03-01
69966,3424,Уфа,127544гвс,NaN,"г Уфа, ул. Рихарда Зорге, д.70 корп.1-2",Многоквартирный дом,2023-03-01,65.387,2023-02-01


In [9]:
data["Адрес объекта"].nunique()

4278

* Тип строения, этажность, площадь, год постройки

In [10]:
print(f"buildings.shape = {buildings.shape}\n")
buildings.sample(2)

buildings.shape = (5873, 5)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
2829,"г Уфа, ул. Комсомольская, д.26",Другое строение,1,NaN,0.01
4289,"г Уфа, ул. Революционная, д.32",Многоквартирный дом,9,1965-01-01 00:00:00,3017.30


In [11]:
buildings["Адрес объекта"].nunique()

5695

* Температуры, продолжительность отопительного периода.

  * ОЗП — осенне-зимний период, время, когда работает система отопления.

In [12]:
temperature = pd.read_excel(
    f'{folder_path}Температуры, продолжительность ОП.xls', index_col=1, skiprows=1
).T[1:].reset_index()

temperature

Период,index,"Тн.в, град.С","Продолжительность ОЗП, сут."
0,2021-07-01,NaN,NaN
1,2021-08-01,NaN,NaN
2,2021-09-01,NaN,NaN
3,2021-10-01,6.696552,29.0
4,2021-11-01,-1.000000,31.0
5,2021-12-01,-5.176667,30.0
6,2022-01-01,-11.500000,31.0
7,2022-02-01,-6.735714,28.0
8,2022-03-01,-5.151613,31.0
9,2022-04-01,5.650000,34.0


## 2. Подготовка данных

In [13]:
preprocess = Preprocess()
data, buildings = preprocess.fit_transform(data, buildings)

## 3. EDA

### 3.1 Структура данных в разрезе типов объектов

In [14]:
pd.concat(
    [
        buildings["Тип Объекта"].value_counts(),
        buildings["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head()

,count,proportion
Тип Объекта,,
Многоквартирный дом,3824,0.653229
Другое строение,1710,0.292108
Частный дом,138,0.023574
"Административные здания, конторы",52,0.008883
"Учебное заведение, комбинат, центр",26,0.004441


* На тип ```Многоквартирный дом``` приходится свыше 65% от всех данных.
* Почти 30% объектов приходится на тип ```Другое строение```

### 3.2. Анализ признака ```Общая площадь объектов```

In [15]:
buildings["Общая площадь объекта"].describe([0.25, 0.5, 0.75, 0.9, 0.99])

,Общая площадь объекта
count,5749.000000
mean,3133.932769
std,4413.071117
min,0.000000
25%,0.010000
50%,2494.600000
75%,4059.500000
90%,7588.040000
99%,19867.062400
max,64516.200000


#### 3.2.1 Объекты с Общей площадью <= 0.1

* Структура отсутствующих данных по площади объектов в разрезе типов объектов.

In [16]:
cond = buildings["Общая площадь объекта"] <= 0.1
pd.concat(
    [
        buildings[cond]["Тип Объекта"].value_counts(),
        buildings[cond]["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head()

,count,proportion
Тип Объекта,,
Другое строение,1569,0.901724
"Административные здания, конторы",48,0.027586
Многоквартирный дом,25,0.014368
Дет.ясли и сады,22,0.012644
Школы и ВУЗ,18,0.010345


* У более чем 90% объектов типа ```Другое строение```, на который приходится почти 30% всех объектов, отсутствуют данные по площади объекта.

* На тип ```Многоквартирный дом``` приходится свыше 65% от всех данных. По этому типу нет данных по площади 25 объектов, что меньше 1.5%

#### 3.2.2 Объекты с Общей площадью > 4000

In [17]:
cond = buildings["Общая площадь объекта"] >= 4000
pd.concat(
    [
        buildings[cond]["Тип Объекта"].value_counts(),
        buildings[cond]["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
)

,count,proportion
Тип Объекта,,
Многоквартирный дом,1447,0.989740
"Учебное заведение, комбинат, центр",8,0.005472
"Жилое здание (Гостиница, Общежитие)",2,0.001368
Другое строение,2,0.001368
Нежилой дом,1,0.000684
Казармы и помещения ВОХР,1,0.000684
Общежитие,1,0.000684


* Объекты с Общей площадью > 4000, в основном, многоквартирные дома.

### 3.3. Объекты без Даты постройки

In [18]:
cond = buildings["Дата постройки"].isnull()
pd.concat(
    [
        buildings[cond]["Тип Объекта"].value_counts(),
        buildings[cond]["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head()

,count,proportion
Тип Объекта,,
Другое строение,1251,0.905210
"Административные здания, конторы",41,0.029667
Многоквартирный дом,34,0.024602
Школы и ВУЗ,13,0.009407
Дет.ясли и сады,13,0.009407


* У более чем 90% объектов типа ```Другое строение```, на который приходится почти 30% всех объектов, отсутствует ```Дата постройки```.

* На тип ```Многоквартирный дом``` приходится свыше 65% от всех данных. По этому типу нет даты постройки 34 объектов, что меньше 2.5%

### 3.4. Неуникальные адреса объектов

In [19]:
df = buildings[
    buildings.duplicated(subset=["Адрес объекта", "Тип Объекта"], keep=False)
]
print(f"Всего неуникальных объектов: {df.shape[0]}\n")
df.iloc[:, :-1].head(10)

Всего неуникальных объектов: 145



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
122,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,1,NaT,0.01
123,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,2,NaT,0.01
412,"г Уфа, пр-кт Октября, д.153",Другое строение,0,NaT,0.01
413,"г Уфа, пр-кт Октября, д.153",Другое строение,3,NaT,0.01
487,"г Уфа, пр-кт Октября, д.31",Другое строение,0,1970-01-01,0.01
488,"г Уфа, пр-кт Октября, д.31",Другое строение,5,1970-01-01,0.01
673,"г Уфа, пр-кт Октября, д.91",Другое строение,1,NaT,0.01
674,"г Уфа, пр-кт Октября, д.91",Другое строение,4,NaT,0.02
882,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,12.00
883,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,9.60


* Неуникальные объекты в разрезе типов объектов:

In [20]:
pd.concat(
    [
      df["Тип Объекта"].value_counts(),
      df["Тип Объекта"].value_counts(True),
      df.groupby(["Тип Объекта"])["Адрес объекта"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта
Тип Объекта,,,
Другое строение,116,0.800000,51
Многоквартирный дом,14,0.096552,7
Частный дом,12,0.082759,4
"Учебное заведение, комбинат, центр",3,0.020690,1


* 80% неуникальных объектов приходится на тип ```Другое строение```, около 10% - на тип ```Многоквартирный дом```.

* Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. В случае с типом ```Другое строение``` - в 90 % случаев эти данные отстутствуют.

#### 3.4.1 Объекты типа ```Многоквартирный дом```

In [21]:
df[df["Тип Объекта"] == "Многоквартирный дом"]

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
1583,"г Уфа, ул. Вологодская, д.13",Многоквартирный дом,9,1979-01-01,2022.00,"г Уфа, ул. Вологодская, д.13, Подобъект №984990"
1584,"г Уфа, ул. Вологодская, д.13",Многоквартирный дом,9,1979-01-01,8256.50,"г Уфа, ул. Вологодская, д.13"
1603,"г Уфа, ул. Вологодская, д.20",Многоквартирный дом,12,1978-01-01,655.00,"г Уфа, ул. Вологодская, д.20, extra"
1604,"г Уфа, ул. Вологодская, д.20",Многоквартирный дом,12,1978-01-01,3681.40,"г Уфа, ул. Вологодская, д.20"
2101,"г Уфа, ул. Интернациональная, д.113",Многоквартирный дом,1,1964-01-01,1161.90,"г Уфа, ул. Интернациональная, д.113, extra"
2102,"г Уфа, ул. Интернациональная, д.113",Многоквартирный дом,5,1964-01-01,2563.70,"г Уфа, ул. Интернациональная, д.113"
2397,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,1,1971-01-01,0.01,"г Уфа, ул. Кирова, д.95, Подобъект №46590"
2398,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,5,1971-01-01,3448.80,"г Уфа, ул. Кирова, д.95"
3765,"г Уфа, ул. Нежинская, д.6",Многоквартирный дом,9,1972-01-01,427.70,"г Уфа, ул. Нежинская, д.6, extra"
3766,"г Уфа, ул. Нежинская, д.6",Многоквартирный дом,9,1972-01-01,2544.60,"г Уфа, ул. Нежинская, д.6"


In [22]:
df[df["Тип Объекта"] == "Многоквартирный дом"]["Адрес объекта 2"].tolist()

['г Уфа, ул. Вологодская, д.13, Подобъект №984990',
 'г Уфа, ул. Вологодская, д.13',
 'г Уфа, ул. Вологодская, д.20, extra',
 'г Уфа, ул. Вологодская, д.20',
 'г Уфа, ул. Интернациональная, д.113, extra',
 'г Уфа, ул. Интернациональная, д.113',
 'г Уфа, ул. Кирова, д.95, Подобъект №46590',
 'г Уфа, ул. Кирова, д.95',
 'г Уфа, ул. Нежинская, д.6, extra',
 'г Уфа, ул. Нежинская, д.6',
 'г Уфа, ул. Революционная, д.88, Подобъект №46372',
 'г Уфа, ул. Революционная, д.88',
 'г Уфа, ул. Энтузиастов, д.6, Подобъект №984984',
 'г Уфа, ул. Энтузиастов, д.6']

На 7 неуникальных адресах находится 14 объектов типа ```Многоквартирный дом``` - по 2 на каждый адрес:
  * ```г Уфа, ул. Вологодская, д.13```,
  * ```г Уфа, ул. Вологодская, д.20```,
  * ```г Уфа, ул. Интернациональная, д.113```,
  * ```г Уфа, ул. Кирова, д.95```,
  * ```г Уфа, ул. Нежинская, д.6```,
  * ```г Уфа, ул. Революционная, д.88```,
  * ```г Уфа, ул. Энтузиастов, д.6```.

  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.

#### 3.4.2 Объекты типа ```Частный дом```

In [23]:
df[df["Тип Объекта"] == "Частный дом"].iloc[:, :-1]

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
882,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,12.00
883,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,9.60
884,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,12.60
885,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,16.23
886,"г Уфа, ул. 9 Января, д.12",Частный дом,1,1998-01-01,17.03
1835,"г Уфа, ул. Губкина, д.6",Частный дом,1,1989-01-01,18.30
1836,"г Уфа, ул. Губкина, д.6",Частный дом,1,1989-01-01,33.06
4783,"г Уфа, ул. Садовая, д.17",Частный дом,1,1947-01-01,39.60
4784,"г Уфа, ул. Садовая, д.17",Частный дом,1,1947-01-01,57.80
5561,"г Уфа, ул. Чукотская, д.23",Частный дом,1,1989-01-01,22.20


In [24]:
set(df[df["Тип Объекта"] == "Частный дом"]["Адрес объекта"])  &  set(data['Адрес объекта'])


set()

* Информации об неуникальных объектах типа ```Частный дом``` нет в данных о потреблении теплоэнергии.

#### 3.4.3 Объекты типа ```Учебное заведение, комбинат, центр```

In [25]:
df[df["Тип Объекта"] == "Учебное заведение, комбинат, центр"]

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
4394,"г Уфа, ул. Рихарда Зорге, д.27","Учебное заведение, комбинат, центр",1,1985-01-01,0.01,"г Уфа, ул. Рихарда Зорге, д.27"
4395,"г Уфа, ул. Рихарда Зорге, д.27","Учебное заведение, комбинат, центр",5,1985-01-01,4008.00,"г Уфа, ул. Рихарда Зорге, д.27, № ОДПУ 00111382"
4396,"г Уфа, ул. Рихарда Зорге, д.27","Учебное заведение, комбинат, центр",12,1985-01-01,3226.90,"г Уфа, ул. Рихарда Зорге, д.27, № ОДПУ 00102590"


* 3 объекта типа ```Учебное заведение, комбинат, центр	``` находятся на одном адресе.
* В данных учета потребления теплоэнергии на этот адрес приходится 2 ```№ ОДПУ```.
* Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.

### 3.5. Подобъекты

In [26]:
cond1_a = data["Тип объекта"] == "Многоквартирный дом"
cond1_b = buildings["Тип Объекта"] == "Многоквартирный дом"

addr = sorted(list(set(data[cond1_a]["Адрес объекта"]) - set(buildings[cond1_b]["Адрес объекта"])))
addr

['г Уфа, б-р. Давлеткильдеева, д.18, Подобъект №987152',
 'г Уфа, б-р. Давлеткильдеева, д.5, Подобъект №985759',
 'г Уфа, б-р. Ибрагимова, д.34, Подобъект №983365',
 'г Уфа, б-р. Молодежный, д.2, Подобъект №992132',
 'г Уфа, б-р. Молодежный, д.6 корп.1, Подобъект №983457',
 'г Уфа, б-р. Тухвата Янаби, д.22, Подобъект №351681',
 'г Уфа, пр-кт Октября, д.122, Подобъект №42688',
 'г Уфа, пр-кт Октября, д.174/2, Подобъект №985197',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48042',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48043',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48044',
 'г Уфа, пр-кт Октября, д.60 корп.1, Подобъект №51211',
 'г Уфа, пр-кт Октября, д.78 корп.3, Подобъект №989097',
 'г Уфа, ул. 50 лет СССР, д.2, Подобъект №47840',
 'г Уфа, ул. 8 Марта, д.12 корп.3, Подобъект №49931',
 'г Уфа, ул. Академика Королева, д.2 корп.2, Подобъект №51212',
 'г Уфа, ул. Ахметова, д.324, Подобъект №986621',
 'г Уфа, ул. Баязита Бикбая, д.44, Подобъект №983215',
 'г Уфа, ул. Блюхера, д.17, Подобъе

In [27]:
len(addr)

77

In [28]:
s = 'г Уфа, ул. Энтузиастов, д.14'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Энтузиастов, д.14'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984513'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984848'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984765']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5715,"г Уфа, ул. Энтузиастов, д.14",Многоквартирный дом,18,2018-01-01,27686.6


In [29]:
data[data["Адрес объекта"]=="г Уфа, ул. Энтузиастов, д.14"]["№ ОДПУ"].unique()

array(['278553,278511'], dtype=object)

In [30]:
data[data["Адрес объекта"]=="г Уфа, ул. Энтузиастов, д.14, Подобъект №984765"]["№ ОДПУ"].unique()

array(['18480739'], dtype=object)

* У подобъектов и объектов отличается ```№ ОДПУ```- учет ведется раздельно, при этом площадь - общая.
* Для использования в моделях необходимо включить подобъекты в объекты.

In [31]:
sorted(list(set(data[cond1_a]["Адрес объекта 2"]) - set(buildings[cond1_b]["Адрес объекта 2"])))

['г Уфа, ул. Пархоменко, д.156 корп.Литер 2в',
 'г Уфа, ул. Уфимское шоссе, д.4']

### 3.6. Многоквартирные дома, отсутствующие в базе данных объектов

#### 3.6.1 ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в```

In [32]:
s = 'г Уфа, ул. Пархоменко, д.156'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Пархоменко, д.156 корп.Литер 2в'
 'г Уфа, ул. Пархоменко, д.156 корп.б'
 'г Уфа, ул. Пархоменко, д.156 корп.в']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
3937,"г Уфа, ул. Пархоменко, д.156 корп.А",Многоквартирный дом,25,2021-11-01,10428.70
3938,"г Уфа, ул. Пархоменко, д.156 корп.б",Многоквартирный дом,16,2019-11-01,21415.40
3939,"г Уфа, ул. Пархоменко, д.156 корп.в",Многоквартирный дом,25,2019-11-01,11241.00
3940,"г Уфа, ул. Пархоменко, д.156 корп.г",Многоквартирный дом,25,2022-11-01,10515.70
3941,"г Уфа, ул. Пархоменко, д.156 корп.Д",Многоквартирный дом,16,2021-11-01,9040.71
3942,"г Уфа, ул. Пархоменко, д.156 корп.е",Многоквартирный дом,25,NaT,0.01
3943,"г Уфа, ул. Пархоменко, д.156 корп.ж",Многоквартирный дом,25,NaT,0.10


* ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в``` в ```data``` не соответствует
аналогу в ```buildings```:
  * ```г Уфа, ул. Пархоменко, д.156 корп.А```
  * ```г Уфа, ул. Пархоменко, д.156 корп.г```
  * ```г Уфа, ул. Пархоменко, д.156 корп.Д```

#### 3.6.2 ```г Уфа, ул. Уфимское шоссе, д.4```

In [33]:
s = 'г Уфа, ул. Уфимское шоссе, д.4'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Уфимское шоссе, д.4' 'г Уфа, ул. Уфимское шоссе, д.4 корп.1']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5275,"г Уфа, ул. Уфимское шоссе, д.4 корп.1",Многоквартирный дом,9,1976-01-01,3821.3


In [34]:
data[data["Адрес объекта"]=="г Уфа, ул. Уфимское шоссе, д.4"]["№ ОДПУ"].unique()

array(['102522'], dtype=object)

In [35]:
data[data["Адрес объекта"]=="г Уфа, ул. Уфимское шоссе, д.4 корп.1"]["№ ОДПУ"].unique()

array(['104237'], dtype=object)

* Объекта ```г Уфа, ул. Уфимское шоссе, д.4``` нет. Есть только ```г Уфа, ул. Уфимское шоссе, д.4 корп.1```, но у него другой ```№ ОДПУ```.

#### 3.6.3. ```г Уфа, ул. Владивостокская, д.10 корп.3,4```

In [36]:
s = 'г Уфа, ул. Владивостокская, д.10'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Владивостокская, д.10'
 'г Уфа, ул. Владивостокская, д.10 корп.3,4']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
1554,"г Уфа, ул. Владивостокская, д.10",Многоквартирный дом,14,2015-01-01,23745.65


In [37]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10"]["№ ОДПУ"].unique()

array(['248788/15024285/20101973/23-190725', '20-101973'], dtype=object)

In [38]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10 корп.3,4"]["№ ОДПУ"].unique()

array(['15024285'], dtype=object)

In [39]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10 корп.3,4"].iloc[:, :-1]

,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
98,98,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-04-01
8764,0,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-06-01
11086,22,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-09-01
14343,61,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,2022-03-01,221.453,2022-02-01
23086,22,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-01-01
36211,97,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-11-01
40445,17,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-08-01
45977,87,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-07-01
46971,35,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-05-01
48359,118,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-12-01


* Объект ```г Уфа, ул. Владивостокская, д.10 корп.3,4``` в data должен попасть в ```г Уфа, ул. Владивостокская, д.10``` - корп.3,4 избыточен.

#### 3.6.4 ``` г Уфа, ул. Даута Юлтыя, д.12```

In [40]:
s = 'г Уфа, ул. Даута Юлтыя, д.12'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654'
 'г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655'
 'г Уфа, ул. Даута Юлтыя, д.12']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
1841,"г Уфа, ул. Даута Юлтыя, д.12",Многоквартирный дом,16,2011-01-01,13465.5


In [41]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12"]["№ ОДПУ"].unique()

array(['89612 89617'], dtype=object)

In [42]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654"]["№ ОДПУ"].unique()

array(['89612'], dtype=object)

In [43]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655"]["№ ОДПУ"].unique()

array(['89617'], dtype=object)

* В объекте ```г Уфа, ул. Даута Юлтыя, д.12``` подобъекты следует объединить - разделены несистемно:
  * ```г Уфа, ул. Даута Юлтыя, д.12```,
  * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654```
  * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655```

#### 3.6.5 ```г Уфа, ул. Комсомольская, д.15..```

In [44]:
s = 'г Уфа, ул. Комсомольская, д.15'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Комсомольская, д.159 корп.1'
 'г Уфа, ул. Комсомольская, д.153 корп.1'
 'г Уфа, ул. Комсомольская, д.159' 'г Уфа, ул. Комсомольская, д.15'
 'г Уфа, ул. Комсомольская, д.153' 'г Уфа, ул. Комсомольская, д.157'
 'г Уфа, ул. Комсомольская, д.157 корп.1'
 'г Уфа, ул. Комсомольская, д.15, Подобъект №35951'
 'г Уфа, ул. Комсомольская, д.15..']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
2780,"г Уфа, ул. Комсомольская, д.15",Многоквартирный дом,23,2013-01-01,59310.4
2781,"г Уфа, ул. Комсомольская, д.153",Многоквартирный дом,9,1976-01-01,7698.9
2782,"г Уфа, ул. Комсомольская, д.153 корп.1",Многоквартирный дом,12,1982-01-01,4794.0
2784,"г Уфа, ул. Комсомольская, д.157",Многоквартирный дом,12,1957-01-01,4758.4
2785,"г Уфа, ул. Комсомольская, д.157 корп.1",Многоквартирный дом,9,1976-01-01,3830.4
2786,"г Уфа, ул. Комсомольская, д.159",Многоквартирный дом,9,1976-01-01,10843.5
2787,"г Уфа, ул. Комсомольская, д.159 корп.1",Многоквартирный дом,7,2005-01-01,15365.5


In [45]:
data[data["Адрес объекта"]=="г Уфа, ул. Комсомольская, д.15.."]

,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления,Адрес объекта 2
66028,598,Уфа,224503,NaN,"г Уфа, ул. Комсомольская, д.15..",Многоквартирный дом,NaT,NaN,2022-08-01,"г Уфа, ул. Комсомольская, д.15"


* Объекта ```г Уфа, ул. Комсомольская, д.15..``` нет. Адрес объекта необходимо исправить.

### 3.7. Объекты без данных учета теплоэнергии

In [46]:
def get_uninvoiced_buildings(data, buildings):
  """
  Выявляет объекты без данных учета теплоэнергии в разрезе типов объектов.
  """
  data_addr = data.groupby(["Тип объекта", "Адрес объекта 2"])["Текущее потребление, Гкал"].sum().reset_index()
  df = buildings.merge(
    data_addr,
    how="left",
    left_on=["Тип Объекта", "Адрес объекта 2"],
    right_on=["Тип объекта", "Адрес объекта 2"]
  )
  return df[df["Тип объекта"].isnull()].iloc[:, :-2]

In [47]:
uninvoiced_buildings = get_uninvoiced_buildings(data, buildings)
uninvoiced_buildings.shape

(1650, 6)

In [48]:
pd.concat(
    [
        uninvoiced_buildings["Тип Объекта"].value_counts(),
        uninvoiced_buildings["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head(10)

,count,proportion
Тип Объекта,,
Многоквартирный дом,752,0.455758
Другое строение,652,0.395152
Частный дом,138,0.083636
"Административные здания, конторы",45,0.027273
"Учебное заведение, комбинат, центр",13,0.007879
Школы и ВУЗ,13,0.007879
Гаражи,6,0.003636
"Жилое здание (Гостиница, Общежитие)",5,0.003030
Производственный объект,4,0.002424


In [49]:
s = uninvoiced_buildings[uninvoiced_buildings["Тип Объекта"] == "Многоквартирный дом"]['Общая площадь объекта'].sum() / 1e06
print(f"У объектов типа Многоквартирный дом общей площадью {s: 0.1f} млн. нет данных о выствленных счетах.")

У объектов типа Многоквартирный дом общей площадью  1.1 млн. нет данных о выствленных счетах.


* У 1650 объектов нет данных об учете потребления теплоэнергии. Из них 752 объекта (46%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 40% - на тип ```Другое строение```, около 8.3% - на тип ```Частный дом```.


## 4. Комментарии к данным


### 4.1. Общая площадь объектов

1. У более чем 90% объектов типа ```Другое строение```, на который приходится почти 30% всех объектов, отсутствуют данные по площади объекта.

2. На тип ```Многоквартирный дом``` приходится свыше 65% от всех данных. По этому типу нет данных по площади 25 объектов, что меньше 1.5%

3. Объекты с Общей площадью > 4000, в основном, многоквартирные дома.

### 4.2 Пропуски в дате постройки объекта

1. У более чем 90% объектов типа ```Другое строение``` отсутствует ```Дата постройки```.

2. По типу ```Многоквартирный дом``` нет даты постройки 34 объектов, что меньше 2.5%

### 4.3 Неуникальные адреса объектов

1. В данных 145 неуникальных объекта, из которых 80% приходится на тип ```Другое строение```, около 10% - на тип ```Многоквартирный дом```, 8% - ```Частный дом```.

  * Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. В случае с типом ```Другое строение``` - в 90 % случаев эти данные отстутствуют.

2. На 7 неуникальных адресах находится 14 объектов типа ```Многоквартирный дом``` - по 2 на каждый адрес:
  * ```г Уфа, ул. Вологодская, д.13```,
  * ```г Уфа, ул. Вологодская, д.20```,
  * ```г Уфа, ул. Интернациональная, д.113```,
  * ```г Уфа, ул. Кирова, д.95```,
  * ```г Уфа, ул. Нежинская, д.6```,
  * ```г Уфа, ул. Революционная, д.88```,
  * ```г Уфа, ул. Энтузиастов, д.6```.

  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.

3. На 4 неуникальных адресах находится 12 объектов типа ```Частный дом```.
  * Ни у одного из 12 объектов данных показаний учёта тепловой энергии нет.
      
4. На 1 неуникальном адресе находятся 3 объекта типа ```Учебное заведение, комбинат, центр```.
  *  В данных учета потребления теплоэнергии на этот адрес приходится 2 ```№ ОДПУ```.
  * Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.


### 4.4 Подобъекты

* У подобъектов и объектов отличается ```№ ОДПУ```- учет ведется раздельно, при этом площадь - общая.
*  Для использования в моделях необходимо включить подобъекты в объекты.

### 4.5. Пропуски в базе данных объектов

Многоквартирные дома, отсутствующие в базе данных объектов:
1. ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в``` в ```data``` не соответствует аналогу в ```buildings```:
  * ```г Уфа, ул. Пархоменко, д.156 корп.А```
  * ```г Уфа, ул. Пархоменко, д.156 корп.г```
  * ```г Уфа, ул. Пархоменко, д.156 корп.Д```

2. Объекта ```г Уфа, ул. Уфимское шоссе, д.4``` нет. Есть только ```г Уфа, ул. Уфимское шоссе, д.4 корп.1```, но у него другой ```№ ОДПУ```.

3. Объект ```г Уфа, ул. Владивостокская, д.10 корп.3,4``` в data должен попасть в ```г Уфа, ул. Владивостокская, д.10``` -  ```корп.3,4``` избыточен.

4. В объекте ```г Уфа, ул. Даута Юлтыя, д.12``` подобъекты следует объединить - разделены несистемно:
    * ```г Уфа, ул. Даута Юлтыя, д.12```,
    * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654```
    * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655```
5. Объекта ```г Уфа, ул. Комсомольская, д.15..``` нет. Адрес объекта необходимо исправить.


### 4.6. Отсутствие данных о выставленных счетах

* У 1650 объектов нет данных об учете потребления теплоэнергии. Из них 752 объекта (46%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 40% - на тип ```Другое строение```, около 8.3% - на тип ```Частный дом```.